In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from theano import *
from lasagne.layers import InputLayer, get_output
import lasagne
import lasagne.layers
import theano.tensor as T
import theano
import numpy as np

import re
import random

Couldn't import dot_parser, loading of dot files will not be possible.


In [3]:
class WikipediaReader(object):
    
    title_rg = re.compile('.*<title>(.*)</title>.*')
    link_rg = re.compile('\[\[([^\]]*)\]\]')
    redirect_rg = re.compile('.*<redirect title="(.*)" />')
    not_link_match = re.compile('[^a-zA-Z0-9_]')    
    
    def __init__(self, fname):
        self.fname = fname
        
    def read(self):
        current_page = None
        look_for_next_page = True
        page_text = None

        title_rg = self.title_rg
      
        with open(self.fname) as f:
            try:
                while True:
                    line = f.next()
                    if look_for_next_page:
                        if '<page>' not in line:
                            continue
                        else:
                            look_for_next_page = False
                    if '<title>' in line:
                        current_page = title_rg.match(line).group(1)
                    elif '<redirect' in line:
                        redirect_page = self.redirect_rg.match(line).group(1)
                        self.readRedirect(current_page, redirect_page)
                        look_for_next_page = True
                    elif '<text' in line:
                        lines = [ line[line.index('>')+2:] ]
                        if '</text>' in lines[0]:
                            page_text = lines[0][:lines[0].index('</text>')]
                            look_for_next_page = True
                            self.readPage(current_page, page_text)
                        else:
                            while True:
                                line = f.next()
                                if '</text>' in line:
                                    lines.append(line[:line.index('</text>')])
                                    look_for_next_page = True
                                    page_text = '\n'.join(lines)
                                    self.readPage(current_page, page_text)
                                    break
                                else:
                                    lines.append(line)
            except StopIteration as e:
                pass
    
    @classmethod
    def getLinkTargets(cls, content):
        ret = cls.link_rg.findall(content)
        def s(v):
            a = v.split('|')
            pg = a[0].replace(' ', '_').replace('(', '_lrb_').replace(')', '_rrb_').lower()
            pg = cls.not_link_match.sub('', pg)
            txt = a[-1]
            if '://' not in v:
                return pg, txt
        return [a for a in [s(r) for r in ret] if a is not None]
            
    def readPage(self, title, content):
        pass
    
    def readRedirect(self, title, target):
        pass
        


In [4]:
from wordvecs import WordVectors, EmbeddingLayer

#wordvectors = WordVectors(fname='../GoogleNews-vectors-negative300.bin', negvectors=False)
wordvectors = WordVectors(
    fname='/data/matthew/enwiki-20141208-pages-articles-multistream-links-output5.bin',
    redir_fname='/data/matthew/enwiki-20141208-redirects.json',
    negvectors=True,
    sentence_length=200
)

In [5]:
len(wordvectors.vectors)

4850513

In [6]:
org_wvectors = set(wordvectors.vectors.keys())

In [7]:
len(wordvectors.redirects)

6594902

In [8]:
class WikiLinkingExp(WikipediaReader):
    
    wiki_re = [
        (re.compile('&amp;'), '&'),
        (re.compile('&lt;'), '<'),
        (re.compile('&gt;'), '>'),
        (re.compile('<ref[^<]*<\/ref>'), ''),
        (re.compile('<.*?>'), ''),
        (re.compile('\[http:[^\] ]*'), ''),
        (re.compile('\|(thumb|left|right|\d+px)', re.IGNORECASE), ''),
        (re.compile('\[\[image:[^\[\]]*\|', re.IGNORECASE), ''),
        (re.compile('\[\[category:([^|\]]*)[^]]*\]\]', re.IGNORECASE), '\\1'),
        (re.compile('\[\[[a-z\-]*:[^\]]\]\]'), ''),
        (re.compile('\[\[[^\|\]]*\|'), '[['),
        (re.compile('{{[^}]*}}'), ''),
        (re.compile('{[^}]*}'), ''),
        (re.compile('[\[|\]]'), ''),
        (re.compile('&[^;]*;'), ' '),
        (re.compile('[^a-zA-Z0-9 ]'), ''),
        (re.compile('\n+'), ' ')
        # TODO: clean up some remaining issues with parsing the wiki text
    ]
    
    @classmethod
    def _wikiToText(cls, txt):
        txt = txt.lower()
        for r in cls.wiki_re:
            txt = r[0].sub(r[1], txt)
        return txt
    
    run_training = False
    num_words_to_use = 200  # set by the value set in the word vectors
    batch_size = 20000
    num_negative_samples = 1
    num_words_per_conv = 3
    
    def __init__(self, fname, wordvecs=wordvectors):
        super(WikiLinkingExp, self).__init__(fname)
        self.wordvecs = wordvecs
        self.current_batch = []
        self.page_titles = set()
        self.num_words_to_use = self.wordvecs.sentence_length
        
        # do an inital load of the data
        self.read()
        
        self._setup()
        
        self.train_cnt = 0
        self.train_res = []
        
    def _setup(self):
        self.y_batch = T.ivector('y_labels')
        self.x_words_batch = T.imatrix('x_words')
        self.x_links_batch = T.imatrix('y_links')
        
        self.sentence_l = InputLayer((None, self.num_words_to_use), input_var=self.x_words_batch)
        self.link_l = InputLayer((None,1), input_var=self.x_links_batch)
        
        self.embedding_W = theano.shared(self.wordvecs.get_numpy_matrix())
        
        self.sentence_emb_l = EmbeddingLayer(
            self.sentence_l, 
            W=self.embedding_W,
            add_word_params=False,
        )
        
        self.link_emb_l = EmbeddingLayer(
            self.link_l,
            W=self.embedding_W,
            add_word_params=False,
        )
        
        self.sentence_conv_l = lasagne.layers.Conv2DLayer(
            self.sentence_emb_l,
            num_filters=150,
            filter_size=(self.num_words_per_conv, self.wordvecs.vector_size),
            name='conv_sent1',
            nonlinearity=lasagne.nonlinearities.rectify,
        )
        
        self.sentence_pool_l = lasagne.layers.MaxPool2DLayer(
            self.sentence_conv_l,
            name='maxing_sent1',
            pool_size=(self.num_words_to_use - self.num_words_per_conv, 1),
        )
        
        self.combined_l = lasagne.layers.ConcatLayer(
            (lasagne.layers.FlattenLayer(self.link_emb_l), lasagne.layers.FlattenLayer(self.sentence_pool_l),)
        )
        
        self.dropped_l = lasagne.layers.DropoutLayer(
            self.combined_l,
            p=.25,
        )
        
        self.dense1_l = lasagne.layers.DenseLayer(
            self.dropped_l,
            num_units=100,
            name='dens1',
            nonlinearity=lasagne.nonlinearities.rectify,
        )
        
        self.dropped2_l = lasagne.layers.DropoutLayer(
            self.dense1_l,
            p=.25
        )
        
        self.out_l = lasagne.layers.DenseLayer(
            self.dropped2_l,
            num_units=2,
            name='dens2',
            nonlinearity=lasagne.nonlinearities.rectify,
        )
        
        self.output_vec = lasagne.layers.get_output(self.out_l)
        self.result_vec = self.output_vec[:,0] - self.output_vec[:,1]
        self.loss_vec = T.nnet.binary_crossentropy(T.clip(self.result_vec + .5, .001, .999), self.y_batch)
        self.output_diff = T.neq(self.result_vec > 0, self.y_batch > .5)
        
        self.all_params = lasagne.layers.get_all_params(self.out_l)
        self.updates = lasagne.updates.adagrad(self.loss_vec.mean(), self.all_params, .01)  # TODO: variable learning rate??
        
        self.train_func = theano.function(
            [self.x_words_batch, self.x_links_batch, self.y_batch],
            [self.loss_vec.sum(), self.output_diff.sum(), self.loss_vec.mean(), self.loss_vec],
            updates=self.updates
        )
        
        self.loss_func = theano.function(
            [self.x_words_batch, self.x_links_batch, self.y_batch],
            [self.loss_vec.sum(), self.loss_vec, self.output_diff.sum()]
        )
        
        
    def train(self):
        self.run_training = True
        self.loss_sum = 0.0
        self.diff_sum = 0
        self.sample_cnt = 0
        self.current_batch = []
        
        self.read()
        if len(self.current_batch) > 0:
            self.train_batch()
        
        r = self.train_cnt, float(self.loss_sum) / self.sample_cnt, float(self.diff_sum) / self.sample_cnt
        self.train_cnt += 1
        self.train_res.append(r)
        return r
        
    def readPage(self, title, content):
        # would be nice to use tf-idf here for the words from the document that should look at, but then won't have that much meanning....
        links = [r[0] for r in self.getLinkTargets(content)]
        words = self._wikiToText(content).split()[:self.num_words_to_use]
        self.d_words = words
        self.d_links = links
        wordsv = self.wordvecs.tokenize(words)
        self.d_wordsv = wordsv
        titlev = self.wordvecs.get_location(title)
        self.d_titlev = titlev
        linksv = self.wordvecs.tokenize(links)
        if self.run_training:
            for l in linksv:
                self.current_batch.append((titlev, wordsv, l, 1))
            for l in random.sample(self.page_titles, len(linksv)*self.num_negative_samples):
                self.current_batch.append((titlev, wordsv, l, 0))
            
            if len(self.current_batch) >= self.batch_size:
                self.train_batch()
        else:
            self.page_titles.add(titlev)
        
    def train_batch(self):
        labels = np.array([r[3] for r in self.current_batch]).astype('int32')
        targets = np.array([[r[2]] for r in self.current_batch]).astype('int32')
        words = np.array([r[1] for r in self.current_batch]).astype('int32')
        
        loss_sum, diff_sum, _, _ = self.train_func(words, targets, labels)
        self.loss_sum += loss_sum
        self.diff_sum += diff_sum
        self.sample_cnt += len(self.current_batch)
        self.current_batch = []


In [9]:
wikiexp = WikiLinkingExp('/data/matthew/enwiki-1e7_lines.xml')

/home/matthew/.virtualenvs/nlp-convnet/lib/python2.7/site-packages/lasagne/init.py:86: UserWarning: The uniform initializer no longer uses Glorot et al.'s approach to determine the bounds, but defaults to the range (-0.01, 0.01) instead. Please use the new GlorotUniform initializer to get the old behavior. GlorotUniform is now the default for all layers.
  warnings.warn("The uniform initializer no longer uses Glorot et al.'s "
/home/matthew/.virtualenvs/nlp-convnet/lib/python2.7/site-packages/lasagne/layers/helper.py:69: UserWarning: get_all_layers() has been changed to return layers in topological order. The former implementation is still available as get_all_layers_old(), but will be removed before the first release of Lasagne. To ignore this warning, use `warnings.filterwarnings('ignore', '.*topo.*')`.
  warnings.warn("get_all_layers() has been changed to return layers in "


In [10]:
len(wikiexp.page_titles)

3921

In [11]:
len(wordvectors.vectors) - 4850513

102947

In [12]:
len(wordvectors.reverse_word_location)

297100

In [13]:
new_words = list(set(wordvectors.vectors) - org_wvectors)

In [14]:
new_words[:200]

['',
 'michael_gross__lrb_actor_rrb_',
 '1850alfheim',
 'clamp__lrb_disambiguation_rrb_',
 'concord1782',
 'filealtair_8800_computerjpg',
 'httpbabylon5warnerbroscom',
 'filesc_oak_oti601_mozartjpg',
 'abc__lrb_the_jackson_5_song_rrb_',
 'battlecruisers6',
 'concord1789',
 'avocationfrom',
 'periodsprehistoric',
 'filered_wine_glassjpg',
 'fileleredoutablephotojpg',
 'parabolic_arc',
 'imagenm142_x_3jpg',
 'truncation__lrb_geometry_rrb_',
 'certified_credit_professional',
 'fileelectron_dotsvg',
 'filebaburnama_illustrationjpg',
 'category1890_deaths',
 'khomska',
 u'college_of_cambridge',
 'categorycaves_of_maharashtra',
 'capitaltheir',
 'filehkljpg',
 u'vim_(text_editor)',
 'berlinoriginating',
 '2002_national_league_division_series',
 'anglobretons',
 'khomsky',
 'james_o_eastland',
 'personequivalence',
 'johannes_palmberg',
 'computerassisted_reviewing',
 u'wolf&#039;s_lair',
 'countrysince',
 'wood2',
 'fileplan_amphipolisensvg',
 'dual__lrb_cognitive_architecture_rrb_',
 'cardi

In [15]:
wordvectors['bentley_mulsanne__lrb_2010_rrb_']

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [ ]:
wikiexp.train()

In [ ]:
for i in xrange(15):
    wikiexp.train()

In [18]:
wikiexp.train_res

[(0, 0.7031445864080539, 0.4959940066309615),
 (1, 0.6932051934312186, 0.49712190767661313),
 (2, 0.6931725602219134, 0.499164116296863),
 (3, 0.6931445327150977, 0.5001160418260647),
 (4, 0.6931527751636032, 0.4997162713593471),
 (5, 0.6931564770285159, 0.4989996174445295),
 (6, 0.693160117523893, 0.4988504208110176)]

In [19]:
wikiexp.sample_cnt

620000

In [18]:
wikiexp.loss_sum / wikiexp.sample_cnt

0.69308540504146954

In [20]:
float(wikiexp.diff_sum) / wikiexp.sample_cnt

0.4994910810810811

In [21]:
len(wordvectors.vectors) - 4850513

2010488

In [23]:
wordvectors.reverse_word_location[:200]

[None,
 'anarchism',
 'is',
 'a',
 'political',
 'philosophy',
 'that',
 'advocates',
 'stateless',
 'societies',
 'often',
 'defined',
 'as',
 'selfgoverned',
 'voluntary',
 'institutions',
 'but',
 'several',
 'authors',
 'have',
 'more',
 'specific',
 'based',
 'on',
 'nonhierarchical',
 'free',
 'associations',
 'holds',
 'the',
 'state',
 'to',
 'be',
 'undesirable',
 'unnecessary',
 'or',
 'harmfulthe',
 'following',
 'sources',
 'cite',
 'while',
 'antistatism',
 'central',
 'entails',
 'opposing',
 'authority',
 'hierarchical',
 'organisation',
 'in',
 'conduct',
 'of',
 'human',
 'relations',
 'including',
 'not',
 'limited',
 'systemas',
 'subtle',
 'and',
 'antidogmatic',
 'draws',
 'many',
 'currents',
 'thought',
 'strategy',
 'does',
 'offer',
 'fixed',
 'body',
 'doctrine',
 'from',
 'single',
 'particular',
 'world',
 'view',
 'instead',
 'fluxing',
 'flowing',
 'there',
 'are',
 'types',
 'traditions',
 'all',
 'which',
 'mutually',
 'exclusive',
 'anarchist',
 'school